In [136]:
import numpy as np
import sys
sys.path.append("../src")
from bert_embedding import *

# Data

## 1. SEC filings

In [4]:
test_link ="https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/test/FIN3"
train_link = "https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt"

In [19]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/test/FIN3.txt -P ../data/sec_ner

In [20]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt -P ../data/sec_ner

## 2. BTC

In [21]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/BTC/CONLL-format/data/h.conll -P ../data/ner_btc

## 3. Wikigold

In [23]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/wikigold/CONLL-format/data/wikigold.conll.txt -P ../data/ner_wikigold

# Data Clean

In [243]:
# load data
def load_ner_data(path, separator = " "):
    with open(path) as f:
        text = f.read().split("\n\n")

    output = []
    for line in text:
        feature_label = []
        line = line.split("\n")
        for entry in line:
            feature_label.append(tuple(entry.split(separator)))
        output.append(feature_label)
    return output

# get words and tags
def unique_words_tags(data):
    unique_words = []
    unique_tags = []
    for sent in data:
        unique_words.extend(list(set(np.array(sent)[:,0])))
        unique_tags.extend(list(set(np.array(sent)[:,-1])))
        
    return set(unique_words), set(unique_tags)

# get words and tags distributions
def distributions_words_tags(data):
    unique_words = {}
    unique_tags = {}
    for i in range(len(data)-1):
        sent = data[i]
        for t in sent:
            word = t[0]
            tag = t[-1]
            
            if word in unique_words:
                unique_words[word] += 1
            else:
                unique_words[word] = 1
                
            if tag in unique_tags:
                unique_tags[tag] += 1
            else:
                unique_tags[tag] = 1
                
    return unique_words, unique_tags

In [244]:
a = []
a.extend(list(set(["1","2","3"])))
a

['3', '1', '2']

In [142]:
sec_path = "../data/ner_sec/FIN5.txt"
sec = load_ner_data(sec_path)

In [143]:
print(len(sec))
# sec[1]

1170


In [144]:
btc = []
for data in ["a", "b", "e", "f", "g", "h"]:
    btc.extend(load_ner_data("../data/ner_btc/" + data + ".conll", "\t"))
print(len(btc))
# btc[1]

9345


In [145]:
wiki = load_ner_data("../data/ner_wikigold/" + "wikigold" + ".conll.txt", " ")
print(len(wiki))
# wiki[1]

1842


In [236]:
wiki[1][0][-1]

'O'

In [245]:
for data in [wiki, sec, btc]:
    print(distributions_words_tags(data)[1])
# only wiki and sec works here

{'I-MISC': 1392, 'O': 32721, 'I-ORG': 1958, 'I-PER': 1634, 'I-LOC': 1447}
{'O': 39485, 'I-ORG': 384, 'I-LOC': 356, 'I-PER': 783, 'I-MISC': 7}
{'O': 131814, 'B-LOC': 2822, 'B-PER': 7928, 'B-ORG': 4135, 'I-ORG': 1176, 'I-PER': 1554, 'I-LOC': 958, '': 5}


In [246]:
for sent in wiki:
    for t in sent:
        if t[-1] =="":
            print(sent)

[('',)]


In [365]:
def sent2labels(sent):
    return [label for token, label in sent]

In [367]:
def sent2labels1(sent):
    return [t[-1] for t in sent]

In [369]:
sent2labels(wiki[0])

['I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O']

In [370]:
sent2labels1(wiki[0])

['I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O']

In [419]:
def word2features(sent, i):
    """
    The function generates all features
    for the word at position i in the
    sentence.
    """
    word = sent[i][0]
    f = tokenize_encode_bert_sentences_sample(tokenizer_d, model_d, word)[0]
    features = {}
    for j in range(len(f)):
        features[str(j)] = f[j]
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [377]:
labels = [sent2labels1(s) for s in wiki]

In [374]:
features = [sent2features(s) for s in wiki]

In [392]:
# features[0:2]

In [393]:
haha = tokenize_encode_bert_sentences_sample(tokenizer_d, model_d, "haha")

In [417]:
len(haha[0])

768

In [427]:
sorted_labels = sorted(
            tags1,
            key=lambda name: (name[1:], name[0])
        )

In [428]:
tags1= {'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}
sorted_labels

['O', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']

In [437]:
a = sec[-2:]
a

[[('/', 'NN', '-', 'O'),
  ('s', 'NNS', '-', 'O'),
  ('/', ':', '-', 'O'),
  ('Bing', 'VBG', '-', 'I-PER'),
  ('Yu', 'NNP', '-', 'I-PER')],
 [('',)]]

In [418]:
haha[0]
haha_f = {}
for j in range(len(haha[0])):
    haha_f[str(j)] = haha[0][j]
haha_f

{'0': -0.2531296,
 '1': -0.03290248,
 '2': -0.19298317,
 '3': -0.46700484,
 '4': -0.23438865,
 '5': -0.022296855,
 '6': 0.34475476,
 '7': 0.24895257,
 '8': -0.20104739,
 '9': -0.15055184,
 '10': -0.02459891,
 '11': -0.046046462,
 '12': -0.16733484,
 '13': 0.2960475,
 '14': 0.023179071,
 '15': 0.15558344,
 '16': -0.11690502,
 '17': 0.27370492,
 '18': 0.25257546,
 '19': -0.19856447,
 '20': -0.019897355,
 '21': -0.3101933,
 '22': 0.020238139,
 '23': -0.27250987,
 '24': -0.06305215,
 '25': 0.039840836,
 '26': -0.023814864,
 '27': 0.0019437447,
 '28': 0.106344394,
 '29': -0.018582601,
 '30': -0.06443295,
 '31': 0.16329055,
 '32': -0.04019996,
 '33': -0.18742768,
 '34': 0.069373414,
 '35': 0.08007872,
 '36': 0.1587943,
 '37': 0.04087455,
 '38': 0.11089118,
 '39': 0.06581404,
 '40': -0.08795494,
 '41': -0.13038576,
 '42': 0.09041588,
 '43': 0.022645438,
 '44': -0.06399354,
 '45': -0.17624572,
 '46': -2.2383463,
 '47': 0.0076247575,
 '48': -0.16515264,
 '49': -0.30726016,
 '50': 0.046568245,
 

In [407]:
word2features(haha, 1)

IndexError: index 1 is out of bounds for axis 0 with size 1

# Model

https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede
https://www.depends-on-the-definition.com/sequence-tagging-lstm-crf/

In [135]:
tokenizer_d = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_d = DistilBertModel.from_pretrained('distilbert-base-uncased')

## 1. LSTM

In [317]:
words_wiki, tags = unique_words_tags(wiki)
words_sec, tags = unique_words_tags(sec)
print(len(words_wiki), len(words_sec), len(words_wiki | words_sec))

8506 3513 10694


In [284]:
sent_lens = [len(x) for x in wiki]
print(max([len(x) for x in wiki]), max([len(x) for x in sec]))

144 413


In [285]:
max_len = 200
word2idx = {w: i + 1 for i, w in enumerate(words_wiki | words_sec)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [286]:
from keras.preprocessing.sequence import pad_sequences
X_wiki = [[word2idx[w[0]] for w in s] for s in wiki]
X_wiki = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y_wiki = [[tag2idx[w[-1]] for w in s] for s in wiki]
y_wiki = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

X_sec = [[word2idx[w[0]] for w in s] for s in sec]
X_sec = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y_sec = [[tag2idx[w[-1]] for w in s] for s in sec]
y_sec = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [291]:
from keras.utils import to_categorical
y_wiki = [to_categorical(i, num_classes=len(tags)) for i in y]
y_wiki[0][0]

y_sec = [to_categorical(i, num_classes=len(tags)) for i in y]
y_sec[0][0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [306]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import keras
from keras import backend as K

sess = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session(sess)
# elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
# elmo_model
# sess.run(tf.global_variables_initializer())
# sess.run(tf.tables_initializer())

In [316]:
# import tensorflow as tf
# from keras.models import Model, Input
# from keras.layers.merge import add
# from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

# # input_text = Input(shape=(768,), dtype=tf.string)
# # embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
# embedding = Input(shape = (768,75))
# x = Bidirectional(LSTM(units=512, return_sequences=True,
#                        recurrent_dropout=0.2, dropout=0.2))(embedding)
# x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
#                            recurrent_dropout=0.2, dropout=0.2))(x)
# x = add([x, x_rnn])  # residual connection to the first biLSTM
# out = TimeDistributed(Dense(len(tags), activation="softmax"))(x)
# model = Model(input_text, out)
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [315]:
# input = Input(shape=(max_len,))
# model = Embedding(input_dim=len(words) + 1, output_dim=20,
#                   input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
# model = Bidirectional(LSTM(units=50, return_sequences=True,
#                            recurrent_dropout=0.1))(model)  # variational biLSTM
# model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
# crf = CRF(len(tags))  # CRF layer
# out = crf(model)  # output

## 2. CRF

In [314]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [349]:
wiki[0]

[('010', 'I-MISC'),
 ('is', 'O'),
 ('the', 'O'),
 ('tenth', 'O'),
 ('album', 'O'),
 ('from', 'O'),
 ('Japanese', 'I-MISC'),
 ('Punk', 'O'),
 ('Techno', 'O'),
 ('band', 'O'),
 ('The', 'I-ORG'),
 ('Mad', 'I-ORG'),
 ('Capsule', 'I-ORG'),
 ('Markets', 'I-ORG'),
 ('.', 'O')]